github link: [czip](https://github.com/DingWB/czip)

In [2]:
import os,sys
%alias czip /anvil/projects/x-mcb130189/Wubin/Software/miniconda3/envs/yap/bin/czip

## Generate 2D subset index for a genomic region
Instead of generating 1D subset index for CGN, CHN, in which, the IDs for the specific pattern will be written into the resulting .ssi file.
2D subset index will create a .ssi file including a genomic region, ID start and ID end will be written.

In [3]:
czip generate_ssi2 --help

INFO: Showing help with the command 'czip generate_ssi2 -- --help'.

NAME
    czip generate_ssi2 - Generate subset index for a genomic region bed file. For example: czip generate_ssi2 -i ~/Ref/mm10/annotations/mm10_with_chrL.allc.cz         -o mm10_with_chrL.allc.genes_flank2k.ssi -b genes_flank2k.bed.gz -n 4

SYNOPSIS
    czip generate_ssi2 INPUT <flags>

DESCRIPTION
    Generate subset index for a genomic region bed file. For example: czip generate_ssi2 -i ~/Ref/mm10/annotations/mm10_with_chrL.allc.cz         -o mm10_with_chrL.allc.genes_flank2k.ssi -b genes_flank2k.bed.gz -n 4

POSITIONAL ARGUMENTS
    INPUT

FLAGS
    -o, --output=OUTPUT
        Type: Optional[]
        Default: None
    -b, --bed=BED
        Type: Optional[]
        Default: None
    -n, --n_jobs=N_JOBS
        Default: 4

NOTES
    You can also use flags syntax for POSITIONAL ARGUMENTS


```shell
czip generate_ssi2 -i ~/Ref/mm10/annotations/mm10_with_chrL.allc.cz \
        -o mm10_with_chrL.allc.genes_flank2k.ssi -b genes_flank2k.bed.gz -n 4
```

## Aggregate genomic regions

In [4]:
czip aggregate --help

INFO: Showing help with the command 'czip aggregate -- --help'.

NAME
    czip aggregate - Aggregate a given genomic region on a .cz file, for example: /usr/bin/time -f "%e    %M      %P" czip aggregate -I test.cz -O test_gene.cz         -s mm10_with_chrL.allc.genes_flank2k.ssi

SYNOPSIS
    czip aggregate <flags>

DESCRIPTION
    Aggregate a given genomic region on a .cz file, for example: /usr/bin/time -f "%e    %M      %P" czip aggregate -I test.cz -O test_gene.cz         -s mm10_with_chrL.allc.genes_flank2k.ssi

FLAGS
    -I, --Input=INPUT
        Type: Optional[]
        Default: None
    -O, --Outfile=OUTFILE
        Type: Optional[]
        Default: None
    -s, --ssi=SSI
        Type: Optional[]
        Default: None
    -i, --intersect=INTERSECT
        Type: Optional[]
        Default: None
    -e, --exclude=EXCLUDE
        Type: Optional[]
        Default: None
    -c, --chunksize=CHUNKSIZE
        Default: 5000
    -f, --formats=FORMATS
        Default: ['H', 'H']


```shell
czip aggregate -I test.cz -O test_gene.cz \
        -s mm10_with_chrL.allc.genes_flank2k.ssi
```

In the resulting test_gene.cz, each row is a region corresponding to one gene, mc and cov are summed up for all CG/CH located at this gene.

## Call DMR pipeline

Let's assume that we have already generated a folder named `cz-CG`, containing CG-only .cz files for each cell. 
If you don't have this folder, it can be generated by `czip bed2cz`, `czip extractCG`

### (1).Merge cell types

```shell
head cell_table.tsv #the 1st column is cell ID and 2nd column is major type
```

```text
FC_P28a_3C_7-3-A11-A17  EC
FC_P0b_3C_2-5-K18-I22   RG-2
FC_P0a_3C_6-5-O21-G10   RG-2
FC_P28b_3C_1-1-M14-I2   IT-L4
FC_P0a_3C_3-6-K15-F23   RG-3
FC_P0a_3C_7-6-C23-F12   RG-4
FC_P0b_3C_5-5-I24-F9    RG-1
FC_P28a_3C_7-5-A11-I10  RG-3
FC_P28b_3C_2-2-K18-K4   ODC
FC_P28b_3C_3-5-K20-E10  MGC
```

```shell
czip merge_cell_type -i cz-CG -c cell_table.tsv 
    -o MajorType/cz -n 96 -P ~/Ref/mm10/mm10_ucsc.main.chrom.sizes
```    

```shell
tree MajorType/cz
```

```shell
MajorType/cz
├── ASC.cz
├── CGE-Lamp5.cz
├── CGE-Vip.cz
├── CT-L6.cz
├── EC.cz
├── ET-L5.cz
├── Exc-Unknown.cz
├── IT-L23.cz
├── IT-L4.cz
├── IT-L6.cz
├── MGC.cz
├── MGE-Pvalb.cz
├── MGE-Sst.cz
├── NP-L6.cz
├── ODC.cz
├── OPC.cz
├── PC.cz
├── RG-1.cz
├── RG-2.cz
├── RG-3.cz
├── RG-4.cz
├── VLMC.cz
└── VLMC-Pia.cz
```

For each CG, the mc and cov in all cells belonging to the same major type were summed up.

### (2).merge multiple .cz files into matrix (fraction, 2D or fisher exact test)

```shell
cd MajorType
mkdir matrix
    
czip merge_cz -i cz -n 96 -f fraction -o matrix/major_type.beta.bed 
    -P ~/Ref/mm10/mm10_ucsc.main.chrom.sizes  
    -r ~/Ref/mm10/annotations/mm10_with_chrL.allCG.forward.cz

czip merge_cz -i cz -n 96 -f fisher -o matrix/major_type.fisher.bed 
    -P ~/Ref/mm10/mm10_ucsc.main.chrom.sizes 
    -r ~/Ref/mm10/annotations/mm10_with_chrL.allCG.forward.cz
```

```shell
tree matrix/
```

```shell
matrix/
├── major_type.beta.bed.gz
├── major_type.beta.bed.gz.tbi
├── major_type.fisher.bed.gz
└── major_type.fisher.bed.gz.tbi
```

```shell
# major_type.beta.bed.gz is the fraction of methylation (mv / cov)
zcat matrix/major_type.beta.bed.gz |les
```

```text
chrom   start   pos     strand  context ASC     CGE-Lamp5       CGE-Vip CT-L6   EC      ET-L5   Exc-Unknown     IT-L23  IT-L4   IT-L6   MGC     MGE-Pvalb       MGE-Sst NP-L6   ODC     OPC     PC      RG-1    RG-2    RG-3    RG-4    VLMC-Pia        VLMC
chr1    3000826 3000827 +       CGT     1.0     0.857   1.0     0.932   0.857   0.889   1.0     0.952   0.972   1.0     0.775   1.0     0.917   1.0     0.909   0.842   1.0     0.882   0.982   0.9     0.947   0.889   0.952
chr1    3001006 3001007 +       CGG     0.889   1.0     0.933   0.982   0.923   0.9     1.0     1.0     0.962   0.944   0.724   1.0     0.933   0.923   0.926   0.9     0.909   0.944   0.921   0.925   0.967   1.0     0.966
chr1    3001017 3001018 +       CGT     0.904   1.0     0.938   0.946   0.786   1.0     1.0     0.983   0.975   1.0     0.737   0.905   0.933   0.929   0.885   0.918   0.75    0.958   0.97    0.795   0.936   1.0     0.913
chr1    3001276 3001277 +       CGA     0.907   1.0     1.0     0.96    0.643   1.0     1.0     0.957   0.946   1.0     0.722   0.773   0.85    0.95    0.826   0.792   0.625   0.944   0.942   0.857   0.894   1.0     0.778
chr1    3001628 3001629 +       CGT     0.857   1.0     0.833   0.939   0.737   1.0     1.0     0.95    0.973   1.0     0.803   0.893   1.0     0.885   0.83    0.754   0.615   0.933   0.936   0.938   0.915   1.0     0.828
chr1    3003225 3003226 +       CGG     0.943   1.0     0.957   0.986   0.958   1.0     1.0     0.962   0.971   0.864   0.847   0.97    1.0     1.0     0.75    0.83    0.909   0.944   0.942   0.967   0.952   0.857   0.808
chr1    3003338 3003339 +       CGC     0.986   1.0     1.0     0.966   0.947   0.952   1.0     0.976   0.991   1.0     0.943   0.913   0.905   1.0     0.917   0.944   0.87    0.976   0.977   0.951   0.961   1.0     1.0
chr1    3003378 3003379 +       CGT     0.797   0.87    0.85    0.897   0.739   0.864   1.0     0.962   0.96    0.909   0.788   0.914   0.931   1.0     0.745   0.72    0.571   0.879   0.895   0.921   0.896   0.833   0.895
chr1    3003581 3003582 +       CGC     0.905   1.0     1.0     0.939   0.882   1.0     1.0     0.948   0.968   0.929   0.954   0.889   1.0     0.923   0.921   0.928   0.909   0.947   0.949   0.98    0.962   1.0     0.815
chr1    3003639 3003640 +       CGG     0.0263  0.182   0.429   0.325   0.167   0.429   0.167   0.246   0.343   0.0833  0.0     0.15    0.4     0.1     0.143   0.0476  0.0     0.348   0.287   0.242   0.191   0.286   0.235
chr1    3003720 3003721 +       CGG     0.816   1.0     0.967   0.96    0.688   0.933   0.75    0.939   0.966   0.778   0.551   0.897   1.0     0.944   0.857   0.838   0.643   0.887   0.932   0.935   0.897   0.6     0.8
chr1    3003884 3003885 +       CGG     0.918   0.867   1.0     0.986   0.895   0.968   0.923   0.966   0.944   0.905   0.819   0.973   1.0     1.0     0.837   0.874   0.481   0.92    0.935   0.959   0.943   0.917   0.854
chr1    3003897 3003898 +       CGG     0.923   0.867   0.857   1.0     0.9     0.968   0.917   0.965   0.968   1.0     0.838   0.946   0.964   1.0     0.87    0.933   0.692   0.925   0.945   0.96    0.927   0.909   0.829
chr1    3004529 3004530 +       CGA     0.925   1.0     0.9     0.956   0.545   1.0     1.0     1.0     0.981   1.0     0.716   1.0     1.0     0.933   0.938   0.806   0.625   0.926   0.972   0.891   0.941   1.0     0.938
chr1    3005997 3005998 +       CGA     0.493   0.8     0.846   0.776   0.526   0.889   0.857   0.805   0.8     0.806   0.544   0.75    0.926   1.0     0.429   0.407   0.357   0.333   0.473   0.507   0.461   0.75    0.533
chr1    3006186 3006187 +       CGC     0.851   0.947   0.944   0.986   0.778   1.0     1.0     0.98    0.981   0.938   0.697   0.95    1.0     1.0     0.78    0.763   0.714   0.884   0.882   0.861   0.89    1.0     0.737
```

```shell
# major_type.beta.fisher.gz is the Fisher's exact test result (whether the given CG/CH is differential methylated in one major cell type against the rest cell types)
zcat matrix/major_type.fisher.bed.gz |les
```

```text
chrom   start   pos     strand  context ASC.odd_ratio   ASC.pval        CGE-Lamp5.odd_ratio     CGE-Lamp5.pval  CGE-Vip.odd_ratio       CGE-Vip.pval    CT-L6.odd_ratio CT-L6.pval      EC.odd_ratio    EC.pval ET-L5.odd_ratio ET-L5.pval      Exc-Unknown.odd>
chr1    3000826 3000827 +       CGT     inf     0.105   0.424   0.383   inf     1.0     0.972   1.0     0.424   0.383   0.567   0.463   inf     1.0     1.42    0.793   2.6     0.225   inf     0.617   0.205   5.5e-06 inf     0.392   0.781   0.564   inf    >
chr1    3001006 3001007 +       CGG     0.543   0.161   inf     1.0     0.984   1.0     3.93    0.257   0.842   0.588   0.63    0.494   inf     1.0     inf     0.0287  1.81    0.476   1.2     1.0     0.143   1.96e-09        inf     0.63    0.984   1.0    >
chr1    3001017 3001018 +       CGT     0.743   0.584   inf     1.0     1.21    1.0     1.44    0.793   0.287   0.0792  inf     1.0     inf     1.0     4.75    0.12    3.23    0.117   inf     0.631   0.185   1.32e-07        0.758   0.665   1.12    1.0    >
chr1    3001276 3001277 +       CGA     1.15    1.0     inf     0.391   inf     0.157   2.89    0.159   0.205   0.0109  inf     1.0     inf     1.0     2.72    0.106   2.18    0.0751  inf     0.25    0.27    6.66e-06        0.39    0.0721  0.66    0.459  >
chr1    3001628 3001629 +       CGT     0.606   0.213   inf     1.0     0.506   0.192   1.62    0.514   0.281   0.0262  inf     0.613   inf     0.617   2.05    0.155   3.97    0.0102  inf     0.402   0.395   0.00527 0.855   0.741   inf     0.159   0.785  >
chr1    3003225 3003226 +       CGG     1.14    1.0     inf     1.0     1.52    1.0     5.19    0.0853  1.59    1.0     inf     1.0     inf     1.0     1.75    0.479   2.43    0.102   0.428   0.164   0.352   0.00228 2.22    0.719   inf     0.621   inf    >
chr1    3003338 3003339 +       CGC     2.53    0.512   inf     1.0     inf     1.0     1.03    1.0     0.647   0.493   0.72    0.528   inf     1.0     1.47    1.0     4.13    0.175   inf     1.0     0.57    0.225   0.371   0.19    0.335   0.165   inf    >
chr1    3003378 3003379 +       CGT     0.539   0.0458  0.941   0.758   0.798   0.729   1.25    0.725   0.394   0.0552  0.893   0.747   inf     0.612   3.78    0.00345 3.61    0.00177 1.42    1.0     0.496   0.00718 1.52    0.794   1.92    0.569   inf    >
chr1    3003581 3003582 +       CGC     0.491   0.132   inf     1.0     inf     1.0     0.823   0.736   0.399   0.216   inf     1.0     inf     1.0     0.991   1.0     1.67    0.623   0.7     0.524   1.12    1.0     0.425   0.235   inf     1.0     0.646  >
chr1    3003639 3003640 +       CGG     0.0961  0.0019  0.823   1.0     2.83    0.0901  1.84    0.112   0.741   1.0     2.81    0.169   0.741   1.0     1.23    0.449   2.05    0.0093  0.334   0.478   0.0     0.000102        0.65    0.782   2.5     0.233  >
chr1    3003720 3003721 +       CGG     0.574   0.0702  inf     0.243   3.93    0.247   3.32    0.0276  0.289   0.0318  1.88    1.0     0.4     0.395   2.12    0.0763  4.08    0.00166 0.463   0.153   0.135   4.21e-16        1.16    1.0     inf     0.257  >
chr1    3003884 3003885 +       CGG     0.937   0.835   0.545   0.328   inf     0.396   6.29    0.0406  0.714   0.655   2.56    0.508   1.01    1.0     2.46    0.151   1.47    0.355   0.799   0.676   0.346   0.000195        3.08    0.358   inf     0.264  >
chr1    3003897 3003898 +       CGG     0.926   0.834   0.5     0.294   0.459   0.188   inf     0.0203  0.694   0.651   2.35    0.722   0.851   0.593   2.2     0.205   2.43    0.0498  inf     0.411   0.367   0.000832        1.36    1.0     2.11    0.717  >
chr1    3004529 3004530 +       CGA     0.983   1.0     inf     1.0     0.721   0.54    1.76    0.769   0.0914  0.000656        inf     1.0     inf     1.0     inf     0.0185  4.42    0.0184  inf     0.631   0.162   1.24e-08        inf     0.251   inf    >
chr1    3005997 3005998 +       CGA     0.763   0.28    3.2     0.069   4.43    0.00246 2.86    7.82e-05        0.88    0.819   6.43    0.00369 4.78    0.142   3.43    2.09e-06        3.45    2.9e-10 3.35    0.00211 0.942   0.771   2.42    0.0151  10.1   >
chr1    3006186 3006187 +       CGC     0.75    0.325   2.41    0.717   2.27    0.713   9.5     0.00223 0.46    0.121   inf     0.103   inf     1.0     6.98    0.000414        7.52    4.5e-06 2.01    0.576   0.277   7.07e-08        2.56    0.222   inf    >
chr1    3006415 3006416 +       CGA     1.35    1.0     inf     1.0     inf     1.0     inf     0.166   0.703   0.524   0.45    0.39    0.35    0.326   0.922   0.755   1.3     0.813   inf     1.0     0.208   3.24e-05        1.27    1.0     inf     1.0    >
chr1    3006781 3006782 +       CGG     1.31    0.8     0.369   0.26    0.574   0.425   1.3     0.818   1.24    1.0     inf     0.589   1.24    1.0     0.451   0.0194  0.888   0.735   0.743   0.589   0.682   0.318   0.742   0.662   0.993   1.0     2.77   >
chr1    3006880 3006881 +       CGG     0.284   2.46e-05        1.35    1.0     8.8     0.00918 5.24    3.54e-05        0.38    0.125   6.01    0.0664  1.35    1.0     1.08    0.874   2.25    0.00235 3.8     0.273   0.245   7.66e-08        1.63    0.479  >
chr1    3007169 3007170 +       CGG     0.388   0.00717 0.84    0.687   inf     0.242   4.6     0.0189  0.687   0.472   inf     0.626   1.17    1.0     0.714   0.254   2.36    0.0416  2.48    0.719   0.616   0.119   1.35    0.79    0.971   1.0     inf    >
chr1    3007429 3007430 +       CGG     0.454   0.00577 0.78    0.673   6.64    0.0277  1.44    0.408   0.465   0.182   3.15    0.488   inf     0.374   1.76    0.109   1.19    0.616   3.95    0.218   1.86    0.0564  1.4     0.779   0.714   0.528   1.57
```

### (3).Call DMR using comb-p

```shell
czip combp -i matrix/major_type.fisher.bed.gz -o DMR_cpv -n 96
```